In [24]:
def RandomForest(a,b):
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.impute import SimpleImputer#填补缺失的类
    from sklearn.ensemble import RandomForestRegressor
    x_full = a     #x.shape = (506, 13)，共有506个样本，每个样本有13个特征
    y_full = b
    n_samples = 1596    #样本数量
    n_features = 7  #标签数量
    x_full = pd.DataFrame(x_full)
    x_missing_reg = x_full.copy()	#随机森林的填补的结果放在x_missing_reg中
    
    #使用np.argsort将缺失值从少到多进行排序
    sortindex = np.argsort(x_missing_reg.isnull().sum(axis=0)).values
    print (sortindex)
    for i in sortindex:
        #构建新特征矩阵（没有被选中去填充的特征+原始的标签）和新标签（被选中去填充的特征）
        df = x_missing_reg
        fillc = df.iloc[: , i]
        df = pd.concat([df.iloc[: , df.columns != i] , pd.DataFrame(y_full)] , axis=1)

        #在新特征矩阵中，对含有缺失值的列，进行0的填补
        df_0 = SimpleImputer(missing_values=np.nan , strategy="constant" , fill_value=0).fit_transform(df)
        
        #构建新的训练集和测试集
        y_train = fillc[fillc.notnull()]#是被选中要填充的值，存在的那些值，非空值
        y_test = fillc[fillc.isnull()]#是被选中的要填充给的值，不存在的那些值，是空值
        x_train = df_0[y_train.index,:]#在新特征矩阵中，我们需要非空值所对应的的索引
        x_test = df_0[y_test.index,:]#空值所对应的记录
        #print(x_test)
        #print(y_test)

        #用随机森林填补缺失值
        rfc = RandomForestRegressor(n_estimators=100)#实例化
        rfc = rfc.fit(x_train,y_train)  #导入训练集去进行训练
        y_predict = rfc.predict(x_test) #用predict接口将x_test导入，得到我们的预测结果，此结果就是要用来填补空值的
        #print(y_predict)
        #将填补好的特征返回到我们的原始的特征矩阵中
        x_missing_reg.loc[x_missing_reg.iloc[:,i].isnull() , i] = y_predict
        #print (x_missing_reg)
    return x_missing_reg
def seek_strange(data):
    #把奇异值标为nan
    for i in range(len(data)):
        for j in range(len(data[i])):
            if (data[i][j]<=0)|(data[i][j]>=1):
                data[i][j]=np.nan
    return data
def datacleaning():
    import numpy as np
    a=np.loadtxt('./data/Btrain_input.txt',dtype=np.float64)
    b=np.loadtxt('./data/Output.txt',dtype=np.float64)
    seek_strange(a)
    a=RandomForest(a,b)
    np.savetxt('train_input.csv',a)
datacleaning()


[0 2 1 3 4 6 5]
